# EDS 296 Homework 2 Geospatial Analysis
Author: Haylee Oyler

Github repo: https://github.com/haylee360/eds296-hw2


In [1]:
# Import packages
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import intake

In [2]:
# Open the CMIP6 data catalog, store as a variable
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')

# Convert the catalog to a df for easier access
cat_df = catalog.df

I decided to stick wtih my area of interest from my first assignment because I find this system interesting. So, I will still be looking at the Congo Basin in West Africa. I'll use the same two models that are well suited to documenting climate changes in that region: GFDL-ESM4 and INM-CM5-0. These are the Institute for Numerical Mathematics (INM) climate model version 5 generation 0 and the Geophysical Fluid Dynamics Laboratory earth systems model 4.

However, I will be changing which variable I look at! Last time, I was interested in precipitation, but now I will be looking at surface air temperature (tas).

Similar to last time, I will be using the historical model and the `ssp370` projection. 

In [3]:
# Specify search terms to query catalog for CanESM5 data
# activity_id: Selecting CMIP for historical and ScenarioMIP for future projections
activity_ids = ['ScenarioMIP', 'CMIP'] 

# source_id: Models selected earlier
source_id = ['GFDL-ESM4' ,'INM-CM5-0']

# experiment_id: I chose the historical data and the ssp370 projection as my two time experimental configurations
experiment_ids = ['historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']

# member_id: Changed the ensemble member here because there was more data available
member_id = 'r1i1p1f1'

# table_id: Selecting monthly atmospheric data, which is the table that precipitation is stored in. 
table_id = 'Amon' 

# variable_id: I chose precipitation flux in kg m-2 s-1, it includes both liquid and solid phases.
variable_id = 'tas' 

In [4]:
# Search through catalog, store results in "res" variable
res = catalog.search(activity_id=activity_ids, source_id=source_id, 
                     experiment_id=experiment_ids, table_id=table_id, variable_id=variable_id,
                     member_id=member_id)

# Display data frame associated with results
display(res.df)

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,ScenarioMIP,NOAA-GFDL,GFDL-ESM4,ssp245,r1i1p1f1,Amon,tas,gr1,s3://cmip6-pds/CMIP6/ScenarioMIP/NOAA-GFDL/GFD...,NaN,20180701
1,ScenarioMIP,NOAA-GFDL,GFDL-ESM4,ssp370,r1i1p1f1,Amon,tas,gr1,s3://cmip6-pds/CMIP6/ScenarioMIP/NOAA-GFDL/GFD...,NaN,20180701
2,ScenarioMIP,NOAA-GFDL,GFDL-ESM4,ssp126,r1i1p1f1,Amon,tas,gr1,s3://cmip6-pds/CMIP6/ScenarioMIP/NOAA-GFDL/GFD...,NaN,20180701
3,ScenarioMIP,NOAA-GFDL,GFDL-ESM4,ssp585,r1i1p1f1,Amon,tas,gr1,s3://cmip6-pds/CMIP6/ScenarioMIP/NOAA-GFDL/GFD...,NaN,20180701
4,CMIP,INM,INM-CM5-0,historical,r1i1p1f1,Amon,tas,gr1,s3://cmip6-pds/CMIP6/CMIP/INM/INM-CM5-0/histor...,NaN,20190610
5,ScenarioMIP,INM,INM-CM5-0,ssp370,r1i1p1f1,Amon,tas,gr1,s3://cmip6-pds/CMIP6/ScenarioMIP/INM/INM-CM5-0...,NaN,20190618
6,ScenarioMIP,INM,INM-CM5-0,ssp245,r1i1p1f1,Amon,tas,gr1,s3://cmip6-pds/CMIP6/ScenarioMIP/INM/INM-CM5-0...,NaN,20190619
7,ScenarioMIP,INM,INM-CM5-0,ssp126,r1i1p1f1,Amon,tas,gr1,s3://cmip6-pds/CMIP6/ScenarioMIP/INM/INM-CM5-0...,NaN,20190619
8,ScenarioMIP,INM,INM-CM5-0,ssp585,r1i1p1f1,Amon,tas,gr1,s3://cmip6-pds/CMIP6/ScenarioMIP/INM/INM-CM5-0...,NaN,20190724
9,CMIP,NOAA-GFDL,GFDL-ESM4,historical,r1i1p1f1,Amon,tas,gr1,s3://cmip6-pds/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/...,NaN,20190726


### Model Selection and Cleaning

In [5]:
# Read in the historical data file for INM
hist_inm = xr.open_zarr(res.df['zstore'][4], storage_options={'anon': True})

# Read in the SSP370 data file
ssp370_inm = xr.open_zarr(res.df['zstore'][5], storage_options={'anon': True})

# Read in the historical data file for GFDL
hist_gfdl = xr.open_zarr(res.df['zstore'][9], storage_options={'anon': True})

# Read in the SSP370 data file
ssp370_gfdl = xr.open_zarr(res.df['zstore'][1], storage_options={'anon': True})

In [7]:
# Concatenate historical and future projections for INM
inm = xr.concat([hist_inm, ssp370_inm], dim="time")

# Concatenate historical and future projections for GFDL
gfdl = xr.concat([hist_gfdl, ssp370_gfdl], dim="time")

In [ ]:
# Define function to generate area weights
def weights(dat):
    # Calculate weighting factor = cosine of latitude
    coslat = np.cos(np.deg2rad(dat.lat))
    weight_factor = coslat / coslat.mean(dim='lat')
    
    # Weight all points by the weighting factor
    computed_weight = dat * weight_factor
    
    # Return the set of weights: this has dimension equal to that of the input data
    return computed_weight

## Prep the INM Model

In [ ]:
# Select tas variable, store as xarray DataArray
pr_inm_370 = inm_370['pr']

# Define min/max bounds for region of interest (NYC)
lat_min, lat_max = 9.3, 13.4
lon_min, lon_max = 12, 34

In [ ]:
# Subsetting for INM model
# Define logical mask: True when lat/lon inside the valid ranges, False elsewhere
congo_lat_inm_370 = (pr_inm_370.lat >= lat_min) & (pr_inm_370.lat <= lat_max)
congo_lon_inm_370 = (pr_inm_370.lon >= lon_min) & (pr_inm_370.lon <= lon_max)

# Find points where the mask value is True, drop all other points
pr_congo_inm_370 = pr_inm_370.where(congo_lat_inm_370 & congo_lon_inm_370, drop=True)

# Average over lat, lon dimensions to get a time series
pr_congo_inm_370 = pr_congo_inm_370.mean(dim=["lat", "lon"])


We have two time periods of interest: 
- historical (1850-2015)
- historical plus future (1850-2100)

Let's start with our historical time period since we want to plot that first. Later on, we'll use the historical + future time period

In [ ]:
# Converts calendar to standard format to troublshoot slicing errors
pr_congo_inm_370 = pr_congo_inm_370.convert_calendar('standard', use_cftime=False)

# Select a time period of interest
pr_congo_inm_370 = pr_congo_inm_370.sel(time=slice('1850-01-01', '2015-12-31'))

In [ ]:
# Calculate the annual mean precipitation
annual_mean_inm_370 = pr_congo_inm_370.groupby('time.year').mean()

# Calculate best-fit parameters for the linear polynomial fit of precipitation to year
x_inm_370 = np.polyfit(annual_mean_inm_370.year, annual_mean_inm_370, 1)

# Generate a polynomial object using those best-fit parameters
trend_line = np.poly1d(x_inm_370)  